In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time

In [2]:
categories = ['EDUCATION', 
              'BUSINESS', 
       'DATING', 'SPORTS', 
       'WEATHER', 'FOOD_AND_DRINK', 
       'HEALTH_AND_FITNESS', 
       'MUSIC_AND_AUDIO', 'NEWS_AND_MAGAZINES', 
       'SOCIAL', 'SHOPPING', 
       'PRODUCTIVITY', 'PHOTOGRAPHY', 
       'MEDICAL', 'PARENTING', 'BEAUTY',
       'COMMUNICATION', 'TOOLS']

In [3]:
def get_links(data):
    '''Given html data this will extract all the links'''
    link_list = []
    for link in data.findAll('a'):
        get_link = link.get('href')
        link_list.append(get_link)
    return link_list

In [4]:
def webpage_html(url):
    r = requests.get(url)
    html = r.text
    return html

In [5]:
def google_scraper(categories):
    '''Will cycle through every category and then request the site, pass the soup to 
        get_links and then clean the list and append results to a new list in a dict
        format that has the format of {cat_name : [link list]}'''
    dicty = dict()
    for category in categories:
        url = f'https://play.google.com/store/apps/category/{category}/collection/topselling_free'
        html = webpage_html(url)
        soup = BeautifulSoup(html)
        links = get_links(soup)
        del links[0:107]
        clean = list(set(links))
        new_list = [app for app in clean if app.find('details') != -1]
        dicty[category] = new_list
        time.sleep(3)
    return dicty

In [6]:
def parse(listing_html, criteria):
    result = listing_html.find(class_=re.compile(r'.*%s' % criteria))
    if result:
        return result.text

In [8]:
def get_description(category_dict):
    dicty = dict()
    url = 'https://play.google.com'
    for category, link_list in category_dict.items():
        listy = list()
        for link in link_list:
            html = webpage_html(url+link)
            soup = BeautifulSoup(html)
            html1 = soup.find('div', {'class':"LXrl4c"})
            description = parse(html1, 'DWPxHb')
            time.sleep(1)
            listy.append(description)
        dicty[category] = listy
    return dicty              

In [ ]:
cat = categories
# This gets links of apps in the given categories
stuff = google_scraper(cat)
# This gets the description of the apps given a list of links for the categories
all_desc = get_description(stuff)